In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop non-beneficial ID columns
application_df.drop(['EIN', 'NAME'], axis=1, inplace=True)

In [3]:
# Example: Adjusting the cutoff point for binning
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts().loc[lambda x : x < 800].index
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(application_types_to_replace,'Other')

In [4]:
# Encode categorical variables
classifications_to_replace = application_df['CLASSIFICATION'].value_counts().loc[lambda x : x < 1800].index
application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(classifications_to_replace,'Other')

In [5]:
# Continue with encoding, splitting, and scaling (no changes needed here from the previous script)
application_dummies = pd.get_dummies(application_df)
y = application_dummies['IS_SUCCESSFUL'].values
X = application_dummies.drop(['IS_SUCCESSFUL'], axis=1).values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [6]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
# Step 3: Adjust the Model Architecture for the Second Attempt
nn = tf.keras.models.Sequential()
# Adding more hidden layers and changing activation functions
nn.add(tf.keras.layers.Dense(units=100, activation='relu', input_dim=X_train_scaled.shape[1]))
nn.add(tf.keras.layers.Dense(units=50, activation='relu'))
nn.add(tf.keras.layers.Dense(units=25, activation='relu'))  # Additional hidden layer
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [8]:
# Consider changing the optimizer or its parameters
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
# Increasing epochs based on the first attempt's observation
fit_model = nn.fit(X_train_scaled, y_train, epochs=150)


Epoch 1/150
804/804 [==============================] - 8s 5ms/step - loss: 0.5713 - accuracy: 0.7235
Epoch 2/150
804/804 [==============================] - 5s 6ms/step - loss: 0.5586 - accuracy: 0.7293
Epoch 3/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5555 - accuracy: 0.7308
Epoch 4/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5531 - accuracy: 0.7316
Epoch 5/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5517 - accuracy: 0.7315
Epoch 6/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5516 - accuracy: 0.7318
Epoch 7/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5495 - accuracy: 0.7327
Epoch 8/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5494 - accuracy: 0.7343
Epoch 9/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5488 - accuracy: 0.7343
Epoch 10/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5491 - accura

In [10]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5850 - accuracy: 0.7304 - 502ms/epoch - 2ms/step
Loss: 0.5849531888961792, Accuracy: 0.7303789854049683


In [11]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)



# Export our model to HDF5 file
nn.save('/content/drive/My Drive/Colab Notebooks/Exported HDF5 files/AlphabetSoupCharity_Optimization_Attempt_02.h5')

Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
